# CNV association mapping simulation and analysis workflow

## Pipeline command interface

In [1]:
sos run 20190717_workflow.ipynb -h

usage: sos run 20190717_workflow.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  genome_partition
  susie
  varbvs
  fisher
  mcmc
  mcmc_multichain
  sier
  hybrid
  get_hist
  varbvs_wg
  simulate

Global Workflow Options:
  --cwd output (as path)
  --genotype-file data/deletion.X.gz (as path)
  --phenotype-file data/deletion.y.gz (as path)
                        phenotype
  --name ''
  --blocks-file . (as path)
  --iteration 5000 (as int)
                        MCMC number of iterations
  --tune-prop 0.25 (as float)
                        MCMC ...
  --target-accept 0.98 (as float)
                        MCMC ...
  --n-chain 10 (as int)
                     

## Minimal examples

### Simulate data
```
sos run 20190717_workflow.ipynb simulate \
    --name simulation_0525 \
    --genotype-file /home/min/GIT/cnv-gene-mapping/data/deletion.X.gz \
    --cwd output \
    --sample-size 500 \
    --n-batch 10
```

### Whole genome analysis using `varbvs`
```
sos run 20190717_workflow.ipynb varbvs_wg \
    --name varbvs_0525 --cwd output \
    --genotype-file output/deletion.X_b30.simulation_0525.samples.X.gz \
    --phenotype-file output/deletion.X_b30.simulation_0525.samples.y.gz
```

### Fisher's exact test per gene
```
sos run 20190717_workflow.ipynb fisher \
    --name fisher_0525 --cwd output \
    --genotype-file output/deletion.X_b30.simulation_0525.samples.X.gz \
    --phenotype-file output/deletion.X_b30.simulation_0525.samples.y.gz
```

### MCMC analysis
```
sos run 20190717_workflow.ipynb mcmc \
    --name mcmc_0525 --cwd output \
    --genotype-file output/deletion.X_b30.simulation_0525.samples.X.gz \
    --phenotype-file output/deletion.X_b30.simulation_0525.samples.y.gz
```

### SuSiE analysis
```
sos run 20190717_workflow.ipynb susie \
    --name susie_0525 --cwd output \
    --genotype-file output/deletion.X_b30.simulation_0525.samples.X.gz \
    --phenotype-file output/deletion.X_b30.simulation_0525.samples.y.gz
```

### Single Effect Regression analysis
```
sos run 20190717_workflow.ipynb sier \
    --name sier_0525 --cwd output \
    --genotype-file output/deletion.X_b30.simulation_0525.samples.X.gz \
    --phenotype-file output/deletion.X_b30.simulation_0525.samples.y.gz
```

### varbvs analysis
```
sos run 20190717_workflow.ipynb varbvs \
    --name varbvs_0525 --cwd output \
    --genotype-file output/deletion.X_b30.simulation_0525.samples.X.gz \
    --phenotype-file output/deletion.X_b30.simulation_0525.samples.y.gz
```

### Hybrid approach
```
sos run 20190717_workflow.ipynb hybrid \
    --name hybrid_0525 --cwd output \
    --genotype-file output/deletion.X_b30.simulation_0525.samples.X.gz \
    --phenotype-file output/deletion.X_b30.simulation_0525.samples.y.gz
```

## Global configurations

In [1]:
[global]
parameter: cwd = path("output")
parameter: genotype_file = path("data/deletion.X.gz")
# phenotype
parameter: phenotype_file = path("data/deletion.y.gz")
parameter: name = ""
parameter: blocks_file = path()
# MCMC number of iterations
parameter: iteration = 5000
# MCMC ...
parameter: tune_prop = 0.25
# MCMC ...
parameter: target_accept = 0.98
# MCMC ...
parameter: n_chain = 10
# MCMC ...
# For some reason on RCC cluster more than 1 cores will not work (stuck)
parameter: n_core = 1
# MCMC ...
parameter: reparameterize = False
## alpha = log(p/1-p) is uniform
## lower bound: p = prevalence, when prevalence = 0.05, alpha = -2.94
## upper bound: p = case proportion, when case = control, alpha = 0
# MCMC ...
parameter: prevalence = 0.05
# MCMC ...
parameter: mcmc_seed = 999
# Hyper-parameters for MCMC and for Single Effect Regression
parameter: hyperparam_file = path()
# SuSiE number of effects
parameter: L = 10
# Whole genome PIPs obtained by varbvs using `varbvs_wg` pipeline, used for hybrid pipeline
parameter: varbvs_wg_pip = path()
# cluster job configurations
parameter: mcmc_walltime = "2.5h"
parameter: sier_walltime = "2h"
parameter: job_size = 80

fail_if(len(name) == 0, msg = 'Please specify a name for this analysis using ``--name`` option. This will be used in all relevant output files.')

import os
import numpy as np

## Partition genome to blocks

In [1]:
[genome_partition]
# For simulation: get real deletion/duplication CNV data and its block
# n_gene_in_block: get_hist: 1, simulate: 20~50, analyze: 1
parameter: input_file = str
# output contain 3 files: 1) input data removing columns with all zeros, 2) file containing block start and end matching index in 1), 3) block start and end without reindex
parameter: output_files = list
# minimum number of genes in a block for copy model
# set it to 1 to use "natural blocks" from input data
parameter: n_gene_in_block = int
## col_index=None: no row names, col_index=0: use first column as row names
parameter: col_index = None
input: input_file
output: output_files
task: trunk_workers = 1, trunk_size = job_size, walltime = '30m', mem = '32G', cores = 1, tags = f'{step_name}_{_output[0]:bn}'
python: expand = '${ }', stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout'
    import pandas as pd
    from operator import itemgetter
    from itertools import *
    from collections import OrderedDict
    ## header = 0: input with header
    data = pd.read_csv(${_input:r}, compression = "gzip", sep = "\t", header = 0, index_col = ${col_index})
    genes_dict = OrderedDict([(x, y) for x,y in zip([i for i in range(data.shape[1])], list(data.columns))])
    data.columns = [i for i in range(data.shape[1])]
    data_clean = data.loc[:, (data != 0).any(axis = 0)]
    indices = list(data_clean.columns)
    bound = list()
    i = 0; j = 1; n_0 = len(indices)
    while (j < n_0):
        if indices[j] - indices[i] >= ${n_gene_in_block} and indices[j] - indices[j-1] > 1:
            bound.append([indices[i], indices[j-1]])
            i = j
        j += 1
    bound.append([indices[i], indices[j-1]])
    bound = [item for item in bound if item[1] != 0]
    if bound[-1] == bound[-2]:
        bound = bound[:-1]
    ## original index consistent with actual number of genes
    pd.DataFrame(bound).to_csv(${_output[2]:r}, sep = "\t", header = False, index = False)
    span = [item[1] - item[0] for item in bound]
    bound2 = list()
    start = 0
    for i in span:
        end = start + i
        start = end + 1
        bound2.extend([end, start])
    bound2 = [0] + bound2[:-1]
    bound3 = [bound2[x:x+2] for x in range(0, len(bound2), 2)]
    ## bound3: index start from 0
    pd.DataFrame(bound3).to_csv(${_output[1]:r}, sep = "\t", header = False, index = False)
    data_clean.columns = [genes_dict[key] for key in list(data_clean.columns)]
    data_clean.to_csv(${_output[0]:r}, compression = "gzip", sep = "\t", header = True, index = False)

The output of this step has 3 files:
- "Cleaned" genotype matrix without column names from boundary: (column names are index from start to end in each boundary and are dropped when the data is saved).

        14      15      16      17      18      19      20      21      22      23      93      94    95       96      97
         0       1       1       0       1       1       1       1       0       0       0       0     1        1       1
         
- boundaries corresponding to cleaned data above

- original boundaries

        block_start     block_end
        14      23
        93      97
        164     177
        229     236

In [ ]:
[susie_1, varbvs_1, mcmc_1, mcmc_multichain_1, sier_1, hybrid_1, susie_cnv_1, get_hist_1, susie_cnv_hybrid_1]
input: genotype_file
output: f"{cwd:a}/{_input:bn}.cleaned.gz", f"{cwd:a}/{_input:bn}.block_index.csv", f"{cwd:a}/{_input:bn}.block_index_original.csv"
sos_run('genome_partition', input_file = _input, output_files = _output, n_gene_in_block = 1, col_index = None)

## Gene level Fisher's exact test 

In [ ]:
[fisher_2]
output: f"{_input[0]:n}.fisher.gz"
task: trunk_workers = 1, trunk_size = 1, walltime = '30m', mem = '4G', cores = 1, tags = f'{step_name}_{_output:bn}'
python: expand = '${ }'
    import pandas as pd
    ## use stats.fisher_exact instead of "from fisher import pvalue", because "pvalue" does not generate the constant pvalue, 
    ## for example, 'pvalue(56,6650,0,6706).two_tail' is not more significant than 'pvalue(24,6682,0,6706).two_tail'
    from scipy import stats
    data = pd.read_csv(${_input[0]:r}, compression = "gzip", sep = "\t", header = 0)
    y = pd.read_csv("${phenotype_file}", header = None, names = ["y"])
    xy = pd.concat([y, data], axis = 1, join = 'inner')
    xy1 = xy[xy["y"] == 1]
    n1 = xy1.shape[0]
    xy0 = xy[xy["y"] == 0]
    n0 = xy0.shape[0]
    res = list()
    for i in list(data.columns):
        res.append([i, sum(xy1.loc[:,i]), n1 - sum(xy1.loc[:,i]), sum(xy0.loc[:,i]), n0 - sum(xy0.loc[:,i]), 
                    stats.fisher_exact([[sum(xy1.loc[:,i]), sum(xy0.loc[:,i])], [n1 - sum(xy1.loc[:,i]), n0 - sum(xy0.loc[:,i])]])[1]])
    pd.DataFrame(res).sort_values(by = 5).to_csv(${_output:r}, compression = "gzip", sep = "\t", header = ["gene", "d_c", "d_nc", "nd_c", "nd_nc", "p"], index = False)

## Obtain independent CNV blocks

In [15]:
[susie_2, varbvs_2, mcmc_2, mcmc_multichain_2, sier_2, hybrid_2, susie_cnv_2, susie_cnv_hybrid_2]
## R: fread(${_input:[0]}, select = ${_blocks.replace('_', ':')})
## similar to fine mapping, create 527 folders and save results for each of them
if os.path.isfile(f'{blocks_file:a}'):
    blocks = [tuple(x.strip().split()) for x in open(f'{blocks_file:a}').readlines() if not x.strip().startswith('#')]
else:
    blocks = [tuple(x.strip().split()) for x in open(f'{_input[1]:a}').readlines()]
input: for_each = ['blocks']
output: f"{_input[0]:nn}_block_{_blocks[0]}_{_blocks[1]}/block_{_blocks[0]}_{_blocks[1]}.gz"
task: trunk_workers = 1, trunk_size = job_size, walltime = '2m', mem = '16G', cores = 1, tags = f'{step_name}_{_output:bn}'
python: expand = '${ }'
    import pandas as pd, numpy as np
    import os
    start, end = map(int, ${_blocks})
    data = pd.read_csv(${_input[0]:r}, compression = "gzip", sep = "\t", header = 0)
    data.iloc[:, start:end+1].to_csv(${_output:r}, compression = "gzip", sep = "\t", header = True, index = False)
    if os.path.isfile(${varbvs_wg_pip:r}):
        varbvs_wg_pips = [x for x in open(${varbvs_wg_pip:r}).readlines()][start:(end+1)]
        print(''.join(varbvs_wg_pips), file = open("${_output:n}.varbvs_pip", "w"))

## Method `susie-cnv-hybrid`

In [1]:
[susie_cnv_hybrid_3]
depends: hyperparam_file, R_library('susieR'), R_library('logistf')
output: f"{_input:n}.{name}.pip"
 
fail_if(not os.path.isfile(f"{_input:n}.varbvs_pip"), msg = f"Cannot find ``{_input:n}.varbvs_pip`` for use with hybrid pipeline!")
expected_effects = np.sum(np.loadtxt(f"{_input:n}.varbvs_pip", dtype=np.float64, usecols = 1))
print(expected_effects, file=open(f"{_output:n}.n_effects", 'w'))

task: trunk_workers = 1, trunk_size = job_size, walltime = sier_walltime, mem = '8G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    library('susieR')
    library('logistf')
    # logistic_regression <- function(X, y){
    #   output <- matrix(0,ncol(X),2)
    #   for (j in 1:ncol(X)){
    #     output[j,] <- as.vector(summary(glm(y ~ X[,j], family = "binomial"))$coef[2,1:2])
    #   }
    #   return(list(betahat = output[,1], sebetahat = output[,2]))
    # }
    firth.logit.reg <- function(X,y){
        outputs <- matrix(0,ncol(X),3)
        for (j in 1:ncol(X)){
            fit <- logistf(y ~ X[,j])
            outputs[j,1] <- fit$coefficients[2]
            outputs[j,2] <- sqrt(diag(vcov(fit)))[2]
            outputs[j,3] <- fit$prob[2]
        }
        return(list(betahat = outputs[,1], sebetahat = outputs[,2], p = outputs[,3]))
    }
    ###
    ###
    X <- as.matrix(read.table(gzfile(${_input:r}), header = TRUE))
    y <- as.matrix(read.table("${phenotype_file}"))
    priors <- read.table("${hyperparam_file}")
    
    sumstats    <- firth.logit.reg(X,y)
    R           <- cor(X)
    pi          <- priors[1,1]
    s0          <- priors[3,1]
    null_weight <- (1 - pi)^ncol(X)
    phi         <- sum(y) / length(y)
    var.y       <- 1 / (phi * (1 - phi))
    n           <- nrow(X)

    if (${expected_effects} >= 1){
        res <- susie_rss(R = R, n = n, L = ceiling(${expected_effects}) + 1,
                         bhat = sumstats$betahat,
                         shat = sumstats$sebetahat,
                         var_y = var.y,
                         scaled_prior_variance = s0^2 / var.y,
                         estimate_prior_variance = FALSE,
                         null_weight = null_weight,
                         standardize = FALSE,
                         check_prior = FALSE)
    } else {
        suppressWarnings({
        res <- susie_rss(R = R, n = n, L = 1,
                         bhat = sumstats$betahat,
                         shat = sumstats$sebetahat,
                         var_y = var.y,
                         scaled_prior_variance = s0^2 / var.y,
                         estimate_prior_variance = FALSE,
                         null_weight = null_weight,
                         max_iter = 1,   
                         standardize = FALSE,
                         check_prior = FALSE)
        })
    }

    names(res$pip) = colnames(X)
    saveRDS(res, ${_output:r})

ERROR: Error in parse(text = x, srcfile = src): <text>:1:1: unexpected '['
1: [
    ^


## Method `hybrid`

In [ ]:
[hybrid_3]
output: f"{_input:n}.{name}.pip"
fail_if(not os.path.isfile(f"{_input:n}.varbvs_pip"), msg = f"Cannot find ``{_input:n}.varbvs_pip`` for use with hybrid pipeline!")
expected_effects = np.sum(np.loadtxt(f"{_input:n}.varbvs_pip", dtype=np.float64, usecols = 1))
print(expected_effects, file=open(f"{_output:n}.n_effects", 'w'))
sos_run('mcmc:3' if expected_effects >= 1 else 'sier:3', expected_effects = expected_effects)

## Method `mcmc`

In [ ]:
[mcmc_multichain_3]
depends: hyperparam_file
output: f"{_input:n}.{name}.pip"
## when chain = 3: walltime = 3h, max_walltime = 100, jobsize = 8
## when chain = 1: walltime = 20m, job = 5, max_walltime = 36
## when chain = 5: walltime = 3h, mem = '12G'
task: trunk_workers = 1, trunk_size = job_size, walltime = mcmc_walltime, mem = '12G', cores = n_core, tags = f'{step_name}_{_output:bn}'
python: expand = '${ }', env={'THEANO_FLAGS': f"base_compiledir={_output:d}"}, stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    import numpy as np, pandas as pd, pymc3 as pm, theano.tensor as tt
    import os
    from scipy.special import expit
    priors = np.loadtxt("${hyperparam_file}")
    pi0, mu0, s0 = priors[0], priors[1], priors[2]
    X_complete = pd.read_csv(${_input:r}, compression = "gzip", sep = "\t", header = 0, dtype = float)
    # remove duplicated columns in X but still keep track of the number of occurance and index removed 
    # in order to reconstruct results later
    X, index_reconstruct, dup_counts = np.unique(X_complete.to_numpy(), axis=1, return_inverse=True, return_counts=True)
    y = np.loadtxt("${phenotype_file}", dtype = int)
    case_prop = sum(y) / y.shape[0]
    invlogit = lambda x: 1/(1 + tt.exp(-x))
    upper = np.log(case_prop / (1-case_prop))
    lower = np.log(${prevalence} / (1-${prevalence}))
    model = pm.Model()
    with model:
        xi = pm.Bernoulli('xi', pi0, shape = X.shape[1]) #inclusion probability for each variable
        if ${reparameterize}:
            beta_offset = pm.Normal('beta_offset', mu = 0, sd = 1, shape = X.shape[1])
            alpha_offset = pm.distributions.continuous.Uniform("alpha_offset", lower = -1, upper = 1)
            beta = pm.Deterministic("beta", mu0 + beta_offset * s0) #Prior for the non-zero coefficients
            alpha = pm.Deterministic("alpha", lower + (alpha_offset+1)/2*(upper - lower))
        else:
            beta = pm.Normal('beta', mu = mu0, sd = s0, shape = X.shape[1])
            alpha = pm.distributions.continuous.Uniform("alpha", lower = lower, upper = upper)
        p = pm.math.dot(X, xi * beta) #Deterministic function to map the stochastics to the output
        y_obs = pm.Bernoulli('y_obs', invlogit(p + alpha), observed = y) #Data likelihood
    with model:
        trace = pm.sample(draws = ${iteration}, init='nuts', chains = ${n_chain}, tune = int(${tune_prop} * ${iteration}),
                        nuts = {"target_accept": ${target_accept}},
                        random_seed = ${mcmc_seed}, cores = min(${n_core}, ${n_chain}), progressbar = True)
    # FIXME: dump trace to pkl here, if needed
    # results
    pip = np.apply_along_axis(np.mean, 0, trace['xi'])
    beta = np.apply_along_axis(np.mean, 0, np.multiply(trace["beta"], trace["xi"]))
    beta_given_inclusion = np.apply_along_axis(np.sum, 0, trace['xi'] * trace['beta']) / np.apply_along_axis(np.sum, 0, trace['xi'])
    # reconstruct original results adding back duplicated variables
    pip = pip / dup_counts
    beta = beta / dup_counts
    results = np.vstack((pip, beta, beta_given_inclusion)).T[index_reconstruct,:]
    results = pd.DataFrame(results, columns = ['inclusion_probability', 'beta', 'beta_given_inclusion'])
    results = results.set_index(X_complete.columns)
    results[["inclusion_probability"]].to_csv(${_output:r}, sep = "\t", header = False, index = True)
    results.to_csv("${_output:n}.pip_beta", sep = "\t", header = False, index = True)
    print("intercept: uniform\nn_chain: ${n_chain}\nn_iter: ${iteration}\ntune_prop: ${tune_prop}\nseed: ${mcmc_seed}\ntarget_accept: ${target_accept}", file=open("${_output:n}.yml", 'w'))

Multi-chain MCMC is meant to help and diagnose convergence. It is essentially running MCMC at different starting points and with different seed. However due to limitation of pymc3 we cannot save multi-chain samples in reasonable amount of memory. The function to save them to files (`trace` option in `sample` function with different backends) are very buggy as of now, and could be depercated in the future according to developers.

Here we implement a version to run multiple single chains and combine the results. They will use different seeds and with `start = "nuts"` they should be using different starting points too.

In [ ]:
[mcmc_3]
depends: hyperparam_file
output: f"{_input:n}.{name}.pip"
## when chain = 3: walltime = 3h, max_walltime = 100, jobsize = 8
## when chain = 1: walltime = 20m, job = 5, max_walltime = 36
task: trunk_workers = 1, trunk_size = job_size, walltime = mcmc_walltime, mem = '8G', cores = 1, tags = f'{step_name}_{_output:bn}'
python: expand = '${ }', env={'THEANO_FLAGS': f"base_compiledir={_output:d}"}, stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    import numpy as np, pandas as pd, pymc3 as pm, theano.tensor as tt
    import os
    from scipy.special import expit
    priors = np.loadtxt("${hyperparam_file}")
    pi0, mu0, s0 = priors[0], priors[1], priors[2]
    X_complete = pd.read_csv(${_input:r}, compression = "gzip", sep = "\t", header = 0, dtype = float)
    # remove duplicated columns in X but still keep track of the number of occurance and index removed 
    # in order to reconstruct results later
    X, index_reconstruct, dup_counts = np.unique(X_complete.to_numpy(), axis=1, return_inverse=True, return_counts=True)
    y = np.loadtxt("${phenotype_file}", dtype = int)
    case_prop = sum(y) / y.shape[0]
    invlogit = lambda x: 1/(1 + tt.exp(-x))
    upper = np.log(case_prop / (1-case_prop))
    lower = np.log(${prevalence} / (1-${prevalence}))
    model = pm.Model()
    with model:
        xi = pm.Bernoulli('xi', pi0, shape = X.shape[1]) #inclusion probability for each variable
        if ${reparameterize}:
            beta_offset = pm.Normal('beta_offset', mu = 0, sd = 1, shape = X.shape[1])
            alpha_offset = pm.distributions.continuous.Uniform("alpha_offset", lower = -1, upper = 1)
            beta = pm.Deterministic("beta", mu0 + beta_offset * s0) #Prior for the non-zero coefficients
            alpha = pm.Deterministic("alpha", lower + (alpha_offset+1)/2*(upper - lower))
        else:
            beta = pm.Normal('beta', mu = mu0, sd = s0, shape = X.shape[1])
            alpha = pm.distributions.continuous.Uniform("alpha", lower = lower, upper = upper)
        p = pm.math.dot(X, xi * beta) #Deterministic function to map the stochastics to the output
        y_obs = pm.Bernoulli('y_obs', invlogit(p + alpha), observed = y) #Data likelihood
    # Fit model multiple times
    results = []
    for i in range(${n_chain}):
        with model:
            trace = pm.sample(draws = ${iteration}, init = 'nuts', chains = 1, tune = int(${tune_prop} * ${iteration}),
                              nuts = {"target_accept": ${target_accept}},
                              random_seed = ${mcmc_seed} + i, cores = 1, progressbar = True)
        # FIXME: dump trace to pkl here, if needed
        # results
        pip = np.apply_along_axis(np.mean, 0, trace['xi'])
        beta = np.apply_along_axis(np.mean, 0, np.multiply(trace["beta"], trace["xi"]))
        beta_given_inclusion = np.apply_along_axis(np.sum, 0, trace['xi'] * trace['beta']) / np.apply_along_axis(np.sum, 0, trace['xi'])
        # reconstruct original results adding back duplicated variables
        pip = pip / dup_counts
        beta = beta / dup_counts
        result = np.vstack((pip, beta, beta_given_inclusion)).T[index_reconstruct,:]
        results.append(pd.DataFrame(result, columns = ['inclusion_probability', 'beta', 'beta_given_inclusion']))
    # merge results
    results = sum(results)/len(results)
    results = results.set_index(X_complete.columns)
    results[["inclusion_probability"]].to_csv(${_output:r}, sep = "\t", header = False, index = True)
    results.to_csv("${_output:n}.pip_beta", sep = "\t", header = False, index = True)
    print("intercept: uniform\nn_chain: ${n_chain}\nn_iter: ${iteration}\ntune_prop: ${tune_prop}\nseed: ${mcmc_seed}\ntarget_accept: ${target_accept}", file=open("${_output:n}.yml", 'w'))

bash: expand = True
    rm -rf {_output:d}/compiledir_*

## Method `sier` (single effect regression)

Set `--expected-effects` to `1` to use the original single effect model. Otherwise based on input `--expected-effects` and availablility of `varbvs` based regional PIP information, the output PIP will be weighted by expected number of effects.

In [ ]:
[sier_3]
depends: R_library("data.table"), hyperparam_file
parameter: expected_effects=-9
if expected_effects < 0:
    expected_effects = min(1, np.sum(np.loadtxt(f"{_input:n}.varbvs_pip", dtype=np.float64, usecols = 1))) if os.path.isfile(f"{_input:n}.varbvs_pip") else 1
output: f"{_input:n}.{name}.pip"
task: trunk_workers = 1, trunk_size = job_size, walltime = sier_walltime, mem = '8G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    # Compute useful posterior statistics in a simple "single effect"
    # Bayesian logistic regression, in which y ~ sigmoid(b0 + x*b) and b ~
    # N(mu0,s0), where b0 is the intercept, b is the regression
    # coefficient, and mu0, s0 are the the prior mean and standard
    # deviation of b. (The intercept is assigned a "flat" prior.)
    #
    # Input X is the n x p "data matrix", where n is the number of samples
    # and p is the number of candidate predictors, and input y is the
    # vector of n observed outcomes.
    #
    # Input argument p1 specifies the prior inclusion probabilities; it
    # should be a vector of length p in which p1[i]/sum(p1) gives the
    # prior probability that the jth candidate predictor has a nonzero
    # effect on the outcome, Y.
    #
    # Note that these computations could probably be made faster and more
    # accurate using a fast 2-d numerical integration (quadrature) method.
    bayes.logistic <- function (X, y, p0, mu0, s0) {

      # These two variables define the 2-d grid used to compute the Monte
      # Carlo (importance sampling) estimates.
      b0  <- seq(-10,10,0.1) 
      b   <- seq(-10,10,0.1)

      # Create the 2-d grid.
      out <- expand.grid(b0 = b0,b = b)
      b0  <- out$b0
      b   <- out$b
      rm(out)

      # Get the number of candidate predictors (p) and importance weights (n).
      p <- ncol(X)
      n <- length(b)

      # Initialize storage for the marginal log-likelihoods (logZ) and the
      # posterior means (mu1) and standard deviations (s1) of the
      # coefficients.
      logZ <- rep(0,p)
      mu1  <- rep(0,p)
      s1   <- rep(0,p)

      # Repeat for each candidate predictor.
      for (j in 1:p)  {

        # Compute the log-importance weights, ignoring constant terms. The
        # important weight is a product of the (logistic) likelihood and
        # the (normal) prior. This is the step that requires the most
        # effort.
        logw <- rep(0,n)
        x    <- X[,j]
        for (i in 1:n) {
          u       <- b0[i] + x*b[i]
          logw[i] <- sum((y - 1)*u + logsigmoid(u)) - ((b[i] - mu0)/s0)^2/2
        }

        # Compute the importance sampling estimate of the marginal
        # log-likelihood (up to a constant of proportionality).
        u       <- max(logw)
        logZ[j] <- log(mean(exp(logw - u))) + u

        # Compute the normalized importance weights.
        w <- softmax(logw)

        # Compute the mean and standard deviation of the coefficient.
        mu1[j] <- sum(w*b)
        s1[j]  <- sqrt(sum(w*(b - mu1[j])^2))
      }

      # Compute the posterior inclusion probabilities.
      p1 <- softmax(logZ + log(p0))

      # Output the data frame containing the computed posterior
      # statistics.
      return(data.frame(p1 = p1,mu1 = mu1,s1 = s1))
    }
    # sigmoid(x) returns the sigmoid of the elements of x. The sigmoid
    # function is also known as the logistic link function. It is the
    # inverse of logit(x).
    sigmoid <- function (x) {
      y   <- x
      y[] <- 0
      y[x > -500] <- 1/(1 + exp(-x))
      return(y)
    }

    # logpexp(x) returns log(1 + exp(x)). The computation is performed in a
    # numerically stable manner. For large entries of x, log(1 + exp(x)) is
    # effectively the same as x.
    logpexp <- function (x) {
      y    <- x
      i    <- which(x < 16)
      y[i] <- log(1 + exp(x[i]))
      return(y)
    }

    # Use this instead of log(sigmoid(x)) to avoid loss of numerical precision.
    logsigmoid <- function (x)
      -logpexp(-x)

    # Compute the softmax of vector x in a more numerically prudent manner
    # that avoids overflow or underflow.
    softmax <- function (x) {
      y <- exp(x - max(x))
      return(y/sum(y))
    }
    ###
    ###
    ###
    X <- as.matrix(read.table(gzfile(${_input:r}), header = TRUE))
    X <- scale(X, center = TRUE, scale = FALSE)
    y <- as.matrix(read.table("${phenotype_file}"))
    p <- dim(X)[2]
    priors <- read.table("${hyperparam_file}")
    p0 <- rep(priors[1,1], 1, p)
    print ("Fitting Bayesian logistic regression model ...")
    out <- bayes.logistic(X, y, p0, priors[2,1], priors[3,1])
    print ("Model fitting completed!")
    pip <- out$p1 * ${expected_effects}
    names(pip) <- colnames(X)
    write.table(t(t(pip)), ${_output:r}, sep='\t', col.names=FALSE, quote=FALSE)

## Method `SuSiE`

In [ ]:
[susie_3]
depends: R_library('susieR'), hyperparam_file
parameter: estimate_prior_method = "simple"
parameter: check_null_threshold = 0.1
output: f"{_input:n}.{name}.rds"
task: trunk_workers = 1, trunk_size = job_size, walltime = '5m', mem = '6G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    library(susieR)
    X <- as.matrix(read.table(gzfile(${_input:r}), header = TRUE))
    y <- as.matrix(read.table("${phenotype_file}"))
    storage.mode(X) = 'double'
    storage.mode(y) = 'double'
    
    # load priors from global estimates and fix it
    priors      <- read.table("${hyperparam_file}")
    s0          <- priors[3,1]
    pi          <- priors[1,1]
    null_weight <- (1 - pi)^ncol(X)
    
    # Gao and Min's first version of SuSiE
    #res = susie(X, y, L = ${L}, 
    #            scaled_prior_variance = pve, 
    #            estimate_residual_variance = TRUE,
    #            estimate_prior_variance = TRUE,
    #            check_null_threshold = ${check_null_threshold},
    #            null_weight = null_weight)
    
    # Newly implemented SuSiE
    #res <- susie(X, y, L = ${L},
    #             scaled_prior_variance = (s0^2) / var(y),
    #             null_weight = null_weight,
    #             estimate_prior_variance = FALSE,
    #             standardize = FALSE)
    
    # Run default susie
    res <- susie(X, y, L=${L}, null_weight = null_weight)
    
    
    names(res$pip) = colnames(X)
    saveRDS(res, ${_output:r})

In [ ]:
[mcmc_4, mcmc_multichain_4, sier_4, hybrid_4]
input: group_by = "all"
output: f"{cwd:a}/{phenotype_file:bn}.{name}_pip.gz"
bash: expand = '${ }'
    cat ${_input} | gzip --best > ${_output:r}

## Method `susie_cnv`

In [ ]:
[susie_cnv_3]
depends: R_library('susieR')
parameter: check_null_threshold = 0.1
output: f"{_input:n}.{name}.rds"
task: trunk_workers = 1, trunk_size = job_size, walltime = '5m', mem = '6G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    library('susieR')
    library('logistf')
    
    # Summary statistics by logistic regression
    # logistic_regression <- function(X, y){
    #   output = matrix(0,ncol(X),2)
    #   for (j in 1:ncol(X)){
    #     output[j,] <- as.vector(summary(glm(y ~ X[,j], family = "binomial"))$coef[2,1:2])
    #   }
    #   return(list(betahat = output[,1], sebetahat = output[,2]))
    # }
    firth.logit.reg <- function(X,y){
        outputs <- matrix(0,ncol(X),3)
        for (j in 1:ncol(X)){
            fit <- logistf(y ~ X[,j])
            outputs[j,1] <- fit$coefficients[2]
            outputs[j,2] <- sqrt(diag(vcov(fit)))[2]
            outputs[j,3] <- fit$prob[2]
        }
        return(list(betahat = outputs[,1], sebetahat = outputs[,2], p = outputs[,3]))
    }
    ###
    ###
    ###
    X <- as.matrix(read.table(gzfile(${_input:r}), header = TRUE))
    y <- as.matrix(read.table("${phenotype_file}"))
    priors <- read.table("${hyperparam_file}")
    #storage.mode(X) <- 'double'
    #storage.mode(y) <- 'double'
    
    sumstats    <- firth.logit.reg(X,y)
    R           <- cor(X)
    s0          <- priors[3,1]
    pi          <- priors[1,1]
    null_weight <- (1 - pi)^ncol(X)
    phi         <- sum(y) / length(y)
    var.y       <- 1 / (phi * (1 - phi))
    
    # Version of newly implemented susie-rss
    res <- susie_rss(R = R, n = nrow(X), L = 10,
                     bhat = sumstats$betahat, 
                     shat = sumstats$sebetahat,
                     var_y = var.y,
                     scaled_prior_variance = (s0^2) / var.y,
                     estimate_prior_variance = FALSE,
                     null_weight = null_weight,
                     standardize = FALSE,
                     check_prior = FALSE)
    
    #res <- susie_rss(R = R, n = nrow(X),
    #                 bhat = sumstats$betahat, 
    #                 shat = sumstats$sebetahat, 
    #                 var_y = var(y),
    #                 L = 10,
    #                 scaled_prior_variance = pve*sqrt(nrow(X)),
    #                 estimate_residual_variance = TRUE,
    #                 estimate_prior_variance = TRUE,
    #                 check_null_threshold = ${check_null_threshold},
    #                 null_weight = null_weight)
    
    names(res$pip) = colnames(X)
    saveRDS(res, ${_output:r})

In [ ]:
[susie_4, varbvs_4, logisticsusie_4, susie_cnv_4, susie_cnv_hybrid_4]
input: group_by = "all"
output:  f"{cwd:a}/{phenotype_file:bn}.{name}_pip.gz"
R: expand = '${ }', stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    files = c(${_input:r,})
    pips = c()
    for (i in 1:length(files)){pips = c(pips, readRDS(files[i])$pip)}
    write.table(t(t(pips)), gzfile(${_output:r}), sep='\t', col.names=FALSE, quote=FALSE)

## Method `logisticsusie`

In [ ]:
[logisticsusie_3]
depends: R_library('logisticsusie'), hyperparam_file
parameter: check_null_threshold = 0.1
output: f"{_input:n}.{name}.rds"
task: trunk_workers = 1, trunk_size = job_size, walltime = sier_walltime, mem = '8G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    library('logisticsusie')
    X <- as.matrix(read.table(gzfile(${_input:r}), header = TRUE))
    y <- as.matrix(read.table("${phenotype_file}"))[,1]
    storage.mode(X) = 'double'
    storage.mode(y) = 'double'
    priors = read.table("${hyperparam_file}")
    res = binsusie(X, y, N=1, prior_mean = priors[2,1], prior_variance = priors[3,1]^2, check_null_threshold = ${check_null_threshold})
    names(res$pip) = colnames(X)
    saveRDS(res, ${_output:r})

## Method `varbvs`

In [ ]:
[varbvs_3]
depends: R_library("varbvs")
output: f"{_input:n}.{name}.rds"
task: trunk_workers = 1, trunk_size = job_size, walltime = '5m', mem = '6G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    X <- as.matrix(read.table(gzfile(${_input:r}), header = TRUE))
    y <- as.matrix(read.table("${phenotype_file}"))
    storage.mode(X) = 'double'
    storage.mode(y) = 'double'
    # logodds <- seq(-log10(ncol(X)), 0, length.out = 20)
    fit <- varbvs::varbvs(X, NULL, y, family = "binomial", verbose = FALSE)
    names(fit$pip) = colnames(X)
    saveRDS(fit, ${_output:r})

## Whole genome analysis using `varbvs`

In [ ]:
[varbvs_wg]
depends: R_library("varbvs")
parameter: maximum_prior_inclusion = 0.0
parameter: Rseed = 999
input: genotype_file, phenotype_file
output: f"{cwd:a}/{_input[0]:bnn}.{name}.rds", f"{cwd:a}/{_input[0]:bnn}.{name}.pip", f"{cwd:a}/{_input[0]:bnn}.{name}.hyperparams"
task: trunk_workers = 1, trunk_size = 1, walltime = '6h', mem = '16G', cores = 1, tags = f'{step_name}_{_output:bn}'
python: expand = "${ }"
    import pandas as pd
    data = pd.read_csv(${_input[0]:r}, compression = "gzip", sep = "\t", header = 0)
    data_clean = data.loc[:, (data != 0).any(axis = 0)]
    data_clean.to_csv("${_input[0]:n}.cleaned.gz", compression = "gzip", sep = "\t", header = True, index = False)

R: expand = '${ }'
    set.seed(${Rseed})
    X <- as.matrix(read.table(gzfile("${_input[0]:n}.cleaned.gz"), header = TRUE))
    y <- as.matrix(read.table(${_input[1]:r}))
    storage.mode(X) = 'double'
    storage.mode(y) = 'double'
    if (${maximum_prior_inclusion}>0) {
        n_grid = 20
        q = (1:ceiling(n_grid*${maximum_prior_inclusion})) / n_grid
    }
    
    # Task 1: Fit logistic regression BVSR to obtain hyperparameter estimates for spike slab model, and PIP
    fit = varbvs::varbvs(X, NULL, y, family = "binomial", update.sa = TRUE, update.b0 = TRUE, ${'logodds = log10(q/(1-q)),' if maximum_prior_inclusion > 0 else ''} verbose = FALSE)
    names(fit$pip) <- colnames(X)
    write.table(t(t(fit$pip)), ${_output[1]:r}, sep='\t', col.names=FALSE, quote=FALSE)
    sigmoid = function(x) 1 / (1 + exp(-x))
    pi1 = sigmoid(log(10) * sum(fit$logodds * fit$w))
    mu0 = sum(fit$b0 * fit$w)
    s0 = sqrt(sum(fit$sa * fit$w))
    
    # Task 2: Fit a linear regression BVSR to obtain estimate for prior variance ("PVE") for downstream SuSiE analysis
    fit_lm = varbvs::varbvs(X, NULL, y, family = "gaussian", update.sa = TRUE, update.b0 = TRUE, ${'logodds = log10(q/(1-q)),' if maximum_prior_inclusion > 0 else ''} verbose = FALSE)
    sigmoid = function(x) 1 / (1 + exp(-x))
    # Get posterior PVE and adjust that by number of causal gene
    # Here I use pi1 estimated from logistic BVSR as prior inclusion probability.
    # The same pi1 will be used to set prior null weight in SuSiE
    total_pve = mean(fit_lm$model.pve)
    ## posterior pve
    pve = total_pve / (ncol(X) * pi1)
    # Here I use prior PVE which is sa * sigma
    # pve = sum(fit_lm$sa * fit_lm$w) * sum(fit_lm$sigma * fit_lm$w)
    write.table(c(pi1,mu0,s0,pve), file = ${_output[2]:r}, sep = "\n", row.names = FALSE, col.names = FALSE)
    saveRDS(list(logit=fit, lm=fit_lm), ${_output[0]:r})

bash: expand = True
    rm -f {_input[0]:n}.cleaned.gz

## Histogram for gene count per CNV

In [ ]:
[get_hist_2]
output: f"{_input[0]:n}.{name}.pdf"
python: expand = '${ }'
    import pandas as pd, matplotlib.pyplot as plt
    blocks = pd.read_csv(${_input[1]:r}, sep = "\t", header = None, names = ["start", "end"])
    spans = [j-i+1 for i,j in zip(blocks["start"], blocks["end"])]
    counts = {i: spans.count(i) for i in set(spans) if i != 0}
    fig, ax = plt.subplots(figsize = (8,6))
    plt.bar(list(counts.keys()), list(counts.values()), width = 0.8)
    ax.set_title("Histogram of number of genes in blocks")
    plt.savefig(${_output:r})

## Simulate CNV blocks
For numerical studies in the manuscript. Input has to use all genotypes including zeros, because in copy model based simulation we have to use actual genome positions, and genotype data without removing any genes is a proxy for genotype positions. Here it is not exactly copy model in the context of SNPs where block size can be based on knowledge of human LD block size -- at least 30 genes per CNV block is based on empirical experiences.

In [ ]:
[simulate_1]
parameter: n_gene_in_block = 30
input: genotype_file
output: data = f"{cwd:a}/{_input:bn}.cleaned.gz", 
        boundary = f"{cwd:a}/{_input:bn}_b{n_gene_in_block}.block_index.csv", 
        original_boundary = f"{cwd:a}/{_input:bn}_b{n_gene_in_block}.block_index_original.csv"
sos_run('genome_partition', input_file = _input, output_files = _output, n_gene_in_block = n_gene_in_block, col_index = None)

In [ ]:
[simulate_2]
input: genotype_file, output_from('simulate_1')['original_boundary']
output: data_partitioned = f"{_input[1]:nn}.gz", 
        gene_names = f"{_input[1]:nn}.gene_names.gz"
python: expand = '${ }'
    import pandas as pd
    from collections import OrderedDict
    data = pd.read_csv(${_input[0]:r}, compression = "gzip", header = 0, sep = "\t", low_memory = False)
    genes_dict = OrderedDict([(x, y) for x,y in zip([i for i in range(data.shape[1])], list(data.columns))])
    data.columns = [i for i in range(data.shape[1])]
    data = data.set_index([[i for i in range(data.shape[0])]])
    bound = pd.read_csv(${_input[1]:r}, header = None, sep = "\t")
    bound2 = [[item[0], item[1]] if item[0] == bound.values[-1][0] else [item[0], bound.values[j+1][0]-1] for j, item in enumerate(bound.values)]
    genes_fill = [genes_dict[j] for item in bound2 for j in range(item[0],item[1]+1)]
    ## 
    pd.DataFrame(genes_fill).T.to_csv(${_output[1]:r}, compression = "gzip", sep = "\t", header = False, index = False)
    fill = list()
    for l in range(data.shape[0]):
        fill.append([data.loc[l, k[0]:k[1]].tolist() for k in bound2])
    res = pd.DataFrame(fill)
    # save original data by partitions
    res.to_csv(${_output[0]:r}, compression = "gzip", sep = "\t", header = False, index = False)

## Simulate effect sizes

In [ ]:
[simulate_3]
parameter: shape = 1.4
parameter: scale = 0.6
parameter: beta_method = "normal"
parameter: pi0 = 0.95
parameter: seed = 999
input: output_from('simulate_1')['original_boundary']
output: f"{_input:nn}.{name}.beta"
python: expand = '${ }', stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    import pandas as pd, numpy as np
    bound = pd.read_csv(${_input:r}, header = None, sep = "\t")
    bound2 = [[item[0], item[1]] if item[0] == bound.values[-1][0] else [item[0], bound.values[j+1][0]-1] for j, item in enumerate(bound.values)]
    def logor_gamma(shape, scale, n):
        return np.log(np.random.gamma(shape, scale, n))
    def logor_normal(mean, se, n):
        return np.random.normal(mean, se, n)
    np.random.seed(${seed})
    beta0 = np.log(${prevalence} / (1-${prevalence}))
    beta1s = [x for x in logor_${beta_method}(${shape}, ${scale}, bound2[-1][1] - bound2[0][0] + 1)]
    beta1s = [np.random.binomial(1, 1-${pi0}) * i for i in beta1s]
    pd.DataFrame(beta1s).to_csv(${_output:r}, sep = "\t", header = False, index = False)
    print("shape: ${shape}\nscale: ${scale}\ndistribution: ${beta_method}\npi0: ${pi0}\nseed: ${seed}", file=open("${_output:n}.yml", 'w'))

## Simulate samples using copy model

In [ ]:
[simulate_4]
parameter: sample_size = 100000 # sample size: default 100000, test: 1000
parameter: n_batch = 200 # number of simulated sample for each job, default: 200, test: 20
assert sample_size % n_batch == 0
batches = [x+1 for x in range(n_batch)]
input: output_from('simulate_2')['data_partitioned'], output_from('simulate_3'), for_each = ['batches']
output: f"{_input[1]:n}.batch_cache_dir/batch_{_batches}.X.gz", 
        f"{_input[1]:n}.batch_cache_dir/batch_{_batches}.y",
        f"{_input[1]:n}.batch_cache_dir/batch_{_batches}.case_matched.X.gz", 
        f"{_input[1]:n}.batch_cache_dir/batch_{_batches}.case_matched.y"
task: trunk_workers = 1, trunk_size = job_size, walltime = '10m', mem = '8G', cores = 1, tags = f'{step_name}_{_output[0]:bn}'
python: expand = '${ }'
    import pandas as pd, numpy as np
    import random, itertools, ast
    data = pd.read_csv(${_input[0]:r}, compression = "gzip", header = None, sep = "\t")
    size = int(${sample_size} / ${n_batch})
    random.seed(${_batches})
    samples_genome = list()
    for i in range(size):
        order = random.sample(data.index.tolist(), data.shape[1])
        s = list(itertools.chain(*list(ast.literal_eval(n) for n in np.diag(data.loc[order, :]))))
        samples_genome.append(s)
    samples_genome_df = pd.DataFrame(samples_genome) # row: sample, column: gene
    samples_genome_df.to_csv(${_output[0]:r}, compression = "gzip", sep = "\t", header = False, index = False)
    beta1s = pd.read_csv(${_input[1]:r}, header = None, sep = "\t")
    beta0 = np.log(${prevalence} / (1-${prevalence}))
    logit_y = np.matmul(samples_genome_df.values, beta1s.values) + beta0
    ys_p = np.exp(logit_y) / (1+np.exp(logit_y))
    ys = np.random.binomial(1, ys_p)
    pd.DataFrame(ys).to_csv(${_output[1]:r}, sep = "\t", header = False, index = False)
    indices1 = [i for i,y in enumerate(ys) if y == 1]
    indices0 = np.random.choice([i for i in range(len(ys)) if i not in indices1], size = len(indices1), replace = False).tolist()
    samples_genome_df.iloc[indices1 + indices0, :].to_csv(${_output[2]:r}, compression = "gzip", sep = "\t", header = False, index = False)
    pd.DataFrame([1] * len(indices1) + [0] * len(indices0)).to_csv(${_output[3]:r}, sep = "\t", header = False, index = False)

In [ ]:
[simulate_5]
input: group_by = "all"
output: genotype = f'{_input[0]:dn}.X.unnamed.gz', phenotype = f'{_input[0]:dn}.y.gz'
bash: expand = "${ }"
    zcat ${paths(_input[2::4])} | gzip --best > ${_output[0]}
    cat ${paths(_input[3::4])} | gzip --best > ${_output[1]}

## Add gene names as columns

In [ ]:
[simulate_6]
input: output_from('simulate_5')['genotype'], output_from('simulate_2')['gene_names']
output: f"{_input[0]:nn}.gz"
bash: expand = "${ }"
    zcat ${paths(_input[1])} ${paths(_input[0])} | gzip --best > ${_output}
    rm -f ${paths(_input[0])}